In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.contrib.rnn as rnn
import tensorflow.contrib.layers as layers
from text_loader import TextLoader 

In [2]:
num_layers  = 3
hidden_size = 512
batch_size  = 1
max_length  = 1000

loader = TextLoader("data/hamlet.txt")
vocab_size = len(loader.vocab)

In [3]:
X = tf.placeholder(tf.int32, [None, None])
x_one_hot = tf.one_hot(X, vocab_size)

cells = [rnn.BasicLSTMCell(hidden_size) for _ in range(num_layers)]
cells = rnn.MultiRNNCell(cells, state_is_tuple=True)

initial_state = cells.zero_state(batch_size, tf.float32)
outputs, states = tf.nn.dynamic_rnn(cells, x_one_hot, 
                                    initial_state=initial_state, dtype=tf.float32)

outputs = tf.reshape(outputs, [-1, hidden_size])
logits = layers.fully_connected(outputs, vocab_size,
                                activation_fn=None)
y_softmax = tf.nn.softmax(logits)
pred = tf.argmax(y_softmax, axis=1)
pred = tf.reshape(pred, [batch_size, -1])

In [4]:
sentence = list()
# 시작 글자 생성
sentence += loader.X[:10].tolist()
print("Start with:", "".join([loader.words[char] for char in sentence]))

saver = tf.train.Saver()
sess_config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
with tf.Session(config=sess_config) as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, "checkpoints/char-rnn_30000")
    
    # 매 이터레이션마다 글자 하나씩 생성
    pred_char, state = sess.run([pred, states], feed_dict={X:[sentence[:]]})
    for i in range(max_length):
        # 이전 스텝에 갖고 있는 state 값을 다음 스텝에 넣어줌
        pred_char, state = sess.run([pred, states], 
            feed_dict={X:[[sentence[-1]]], initial_state: state})
        sentence.append(pred_char[0][-1])
        
sentence = [loader.words[char] for char in sentence]
print("".join(sentence))

Start with: <START>	HAMLET


INFO:tensorflow:Restoring parameters from checkpoints/char-rnn_30000
<START>	HAMLET

ACT IV



SCENE III	Another room in the castle.


	[Enter KING CLAUDIUS, QUEEN GERTRUDE, their trains, &c]

	The queen, the courtiers: who is this they follow?
	And with such maimed rites? This doth betoken
	The corse they follow did with desperate hand
	Fordo its own life: 'twas of some estate.
	Couch we awhile, and mark.

	[Retiring with HORATIO]

LAERTES	What ceremony else?

HAMLET	That is Laertes,
	A very noble youth: mark.

LAERTES	What ceremony else?

HAMLET	That is Laertes,
	A very noble youth: mark.

LAERTES	What ceremony else?

HAMLET	That is Laertes,
	A very noble youth: mark.

LAERTES	What ceremony else?

HAMLET	That is Laertes,
	A very noble youth: mark.

LAERTES	What ceremony else?

HAMLET	That is Laertes,
	A very noble youth: mark.

LAERTES	What ceremony else?

HAMLET	That is Laertes,
	A very noble youth: mark.

LAERTES	What ceremony else?

HAMLET	That is Laer